In [2]:
import pandas as pd
import numpy as np
import akshare as ak
import baostock as bs
from utils import get_index_components, get_csindex_components

array([4, 1])

In [28]:
def get_components_history(index_code, qdate):
    return get_csindex_components(index_code)

def get_change_dates(stt_date, end_date, freq='year'):
    if freq=='month':
        target_list = pd.date_range(start=stt_date, end=end_date, freq='M').strftime('%Y-%m-%d').tolist()
    elif freq=='year':
        target_list = pd.date_range(start=stt_date, end=end_date, freq='Y').strftime('%Y-%m-%d').tolist()
    else:
        target_list = pd.date_range(start=stt_date, end=end_date).strftime('%Y-%m-%d').tolist()
    return target_list

def stock_selector(code_list, topk=10):
    return np.random.choice(code_list, tokp, replace=False)


def backtesting(index_code, stt_date, end_date, freq='year'):
    pass
    

In [14]:
def get_dividend(code_list, stt_y, end_y, yearType='report'):
    bs.login()
    rs_list = []
    for code in code_list:
        for y in np.arange(stt_y, end_y+1):
            rs_dividend = bs.query_dividend_data(code=code, year=str(y), yearType=yearType)
            while (rs_dividend.error_code == '0') & rs_dividend.next():
                rs_list.append(rs_dividend.get_row_data())
    bs.logout()
    return pd.DataFrame(rs_list, columns=rs_dividend.fields)
        
get_dividend(['sh.600900','sz.000002'], 2020, 2024)

login success!
logout success!


,code,dividPreNoticeDate,dividAgmPumDate,dividPlanAnnounceDate,dividPlanDate,dividRegistDate,dividOperateDate,dividPayDate,dividStockMarketDate,dividCashPsBeforeTax,dividCashPsAfterTax,dividStocksPs,dividCashStock,dividReserveToStockPs
0,sh.600900,,2020-05-22,2020-04-30,2020-07-13,2020-07-16,2020-07-17,2020-07-17,,0.68,0.612或0.68,0.000000,10派6.8元（含税，扣税后6.12或6.8元）,
1,sh.600900,,2021-06-01,2021-04-30,2021-07-09,2021-07-15,2021-07-16,2021-07-16,,0.7,0.63或0.7,0.000000,10派7元（含税，扣税后6.3或7元）,
2,sh.600900,,2022-05-26,2022-04-30,2022-07-13,2022-07-20,2022-07-21,2022-07-21,,0.8153,0.73377或0.8153,0.000000,10派8.153元（含税，扣税后7.3377或8.153元）,
3,sh.600900,,2023-05-24,2023-04-28,2023-07-17,2023-07-20,2023-07-21,2023-07-21,,0.8533,0.76797或0.8533,0.000000,10派8.533元（含税，扣税后7.6797或8.533元）,
4,sh.600900,,2024-05-24,2024-04-30,2024-07-11,2024-07-18,2024-07-19,2024-07-19,,0.82,0.738或0.82,0.000000,10派8.2元（含税，扣税后7.38或8.2元）,
5,sz.000002,,2020-07-01,2020-03-18,2020-08-08,2020-08-13,2020-08-14,2020-08-14,,1.016613,0.9149518或1.0166131,0.000000,10派10.166131元（含税，扣税后9.149518或10.166131元）,
6,sz.000002,,2021-07-01,2021-03-31,2021-08-18,2021-08-24,2021-08-25,2021-08-25,,1.25,1.125或1.25,0.000000,10派12.5元（含税，扣税后11.25或12.5元）,
7,sz.000002,,2022-06-29,2022-03-31,2022-08-18,2022-08-24,2022-08-25,2022-08-25,,0.976126,0.8785131或0.9761257,0.000000,10派9.761257元（含税，扣税后8.785131或9.761257元）,
8,sz.000002,,2023-07-01,2023-03-31,2023-08-21,2023-08-24,2023-08-25,2023-08-25,,0.68,0.612或0.68,0.000000,10派6.8元（含税，扣税后6.12或6.8元）,


In [9]:
get_csindex_components('000922')


array(['sh.600000', 'sh.600012', 'sh.600015', 'sh.600016', 'sh.600019',
       'sh.600028', 'sh.600039', 'sh.600048', 'sh.600057', 'sh.600064',
       'sh.600123', 'sh.600153', 'sh.600177', 'sh.600188', 'sh.600273',
       'sh.600282', 'sh.600295', 'sh.600325', 'sh.600348', 'sh.600350',
       'sh.600373', 'sh.600377', 'sh.600389', 'sh.600395', 'sh.600398',
       'sh.600461', 'sh.600479', 'sh.600546', 'sh.600585', 'sh.600642',
       'sh.600674', 'sh.600704', 'sh.600741', 'sh.600755', 'sh.600757',
       'sh.600782', 'sh.600801', 'sh.600808', 'sh.600873', 'sh.600900',
       'sh.600901', 'sh.600919', 'sh.600971', 'sh.600985', 'sh.600997',
       'sh.601000', 'sh.601006', 'sh.601009', 'sh.601019', 'sh.601077',
       'sh.601088', 'sh.601098', 'sh.601166', 'sh.601169', 'sh.601216',
       'sh.601225', 'sh.601229', 'sh.601288', 'sh.601328', 'sh.601398',
       'sh.601601', 'sh.601636', 'sh.601666', 'sh.601668', 'sh.601699',
       'sh.601818', 'sh.601857', 'sh.601928', 'sh.601939', 'sh.6

,日期,指数代码,指数名称,指数英文名称,成分券代码,成分券名称,成分券英文名称,交易所,交易所英文名称
0,2024-12-06,000922,中证红利,CSI Dividend,000002,万科A,China Vanke Co Ltd,深圳证券交易所,Shenzhen Stock Exchange
1,2024-12-06,000922,中证红利,CSI Dividend,000090,天健集团,"SHENZHEN TAGEN GROUP CO., LTD.",深圳证券交易所,Shenzhen Stock Exchange
2,2024-12-06,000922,中证红利,CSI Dividend,000157,中联重科,Zoomlion Heavy Industry Science & Technology C...,深圳证券交易所,Shenzhen Stock Exchange
3,2024-12-06,000922,中证红利,CSI Dividend,000581,威孚高科,Weifu High-Technology Group Co Ltd,深圳证券交易所,Shenzhen Stock Exchange
4,2024-12-06,000922,中证红利,CSI Dividend,000651,格力电器,"Gree Electric Appliances,Inc. of Zhuhai",深圳证券交易所,Shenzhen Stock Exchange
...,...,...,...,...,...,...,...,...,...
95,2024-12-06,000922,中证红利,CSI Dividend,603565,中谷物流,"Shanghai Zhonggu Logistics Co., Ltd.",上海证券交易所,Shanghai Stock Exchange
96,2024-12-06,000922,中证红利,CSI Dividend,603878,武进不锈,"JIANG SU WUJIN STAINLESS STEEL PIPE GROUP CO.,...",上海证券交易所,Shanghai Stock Exchange
97,2024-12-06,000922,中证红利,CSI Dividend,603886,元祖股份,"Ganso Co., Ltd.",上海证券交易所,Shanghai Stock Exchange
98,2024-12-06,000922,中证红利,CSI Dividend,605368,蓝天燃气,"Henan Lantian Gas Co.,Ltd.",上海证券交易所,Shanghai Stock Exchange


In [3]:
index_stock_cons_csindex_df = ak.index_stock_cons_csindex(symbol="000300")
index_stock_cons_csindex_df


,日期,指数代码,指数名称,指数英文名称,成分券代码,成分券名称,成分券英文名称,交易所,交易所英文名称
0,2024-12-06,000300,沪深300,CSI 300,000001,平安银行,"Ping An Bank Co., Ltd.",深圳证券交易所,Shenzhen Stock Exchange
1,2024-12-06,000300,沪深300,CSI 300,000002,万科A,China Vanke Co Ltd,深圳证券交易所,Shenzhen Stock Exchange
2,2024-12-06,000300,沪深300,CSI 300,000063,中兴通讯,ZTE Corporation,深圳证券交易所,Shenzhen Stock Exchange
3,2024-12-06,000300,沪深300,CSI 300,000100,TCL科技,TCL Technology Group Corporation,深圳证券交易所,Shenzhen Stock Exchange
4,2024-12-06,000300,沪深300,CSI 300,000157,中联重科,Zoomlion Heavy Industry Science & Technology C...,深圳证券交易所,Shenzhen Stock Exchange
...,...,...,...,...,...,...,...,...,...
295,2024-12-06,000300,沪深300,CSI 300,688303,大全能源,"XinJiang Daqo New Energy Co.,Ltd",上海证券交易所,Shanghai Stock Exchange
296,2024-12-06,000300,沪深300,CSI 300,688363,华熙生物,Bloomage Biotechnology Corporation Limited,上海证券交易所,Shanghai Stock Exchange
297,2024-12-06,000300,沪深300,CSI 300,688396,华润微,China Resources Microelectronics Limited,上海证券交易所,Shanghai Stock Exchange
298,2024-12-06,000300,沪深300,CSI 300,688599,天合光能,"Trina Solar Co., Ltd.",上海证券交易所,Shanghai Stock Exchange


In [24]:
0.82/30.54*100

2.6850032743942367